# 5. Model Selection and 6. Model Training 

### 5. Model Selection

a. Test multiple algorithms, such as Decision Trees, Random Forest, 
Gradient Boosting, or XGBoost. <br>
b. Use cross-validation to assess each model’s robustness and avoid overfitting.<br> 
c. Choose a model that provides the best balance of performance metrics.

In [5]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dataprep_module import load_and_prepare_data
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, classification_report, confusion_matrix, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

# Load data from preprocessing module
print("Loading data from dataprep_module...")
x, y, preprocess = load_and_prepare_data()
print(f"Data loaded: {x.shape[0]} samples, {x.shape[1]} features")
print(f"Fraud rate: {y.mean()*100:.2f}%")

# Set up cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
print(f"Using {cv.n_splits}-fold stratified cross-validation")

# ADD THIS PART - Preprocess data once and store in memory
print("Preprocessing data once and storing in memory...")
x_processed = preprocess.fit_transform(x)
print(f"Processed data shape: {x_processed.shape}")
print(f"Memory usage: ~{x_processed.nbytes / 1024**3:.2f} GB")
print(f"Data types: {type(x_processed)}")
print(f"Sample values: {x_processed[0][:5]}")  # Show first 5 values

Loading data from dataprep_module...
Data loaded: 20499 samples, 14 features
Fraud rate: 1.91%
Using 5-fold stratified cross-validation
Preprocessing data once and storing in memory...


Processed data shape: (20499, 57933)
Memory usage: ~8.85 GB
Data types: <class 'numpy.ndarray'>
Sample values: [0.51637396 0.31874451 0.         0.         0.        ]


In [23]:
# Model 2: Decision Tree
# TO TAKE

print("="*50)
print("MODEL 2: DECISION TREE")
print("="*50)

dt_model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("classifier", DecisionTreeClassifier(
        random_state=42,
        class_weight='balanced',
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=2
    ))
])

scorers_dt = {
    "roc_auc": "roc_auc",
    "accuracy": "accuracy",
    "precision": "precision",
    "recall": "recall",
    "f1": "f1"
}

# Cross-validation scores
dt_auc_scores = cross_validate(
    dt_model, 
    x, y, 
    cv=cv, 
    scoring=scorers_dt)


for m in scorers_dt:
    s = dt_auc_scores[f"test_{m}"]
    print(f"{m.upper():<9}: {s.mean():.3f} (+/- {s.std()*2:.3f})")


MODEL 2: DECISION TREE


ROC_AUC  : 0.640 (+/- 0.031)
ACCURACY : 0.513 (+/- 0.128)
PRECISION: 0.028 (+/- 0.002)
RECALL   : 0.721 (+/- 0.182)
F1       : 0.054 (+/- 0.003)


In [25]:
print("="*50)
print("MODEL 3: HIST-GRADIENT BOOSTING (EARLY STOP)")
print("="*50)

hgb_model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("classifier", HistGradientBoostingClassifier(
        random_state=42,
        max_iter=300,            # like n_estimators
        learning_rate=0.08,
        max_leaf_nodes=31,
        early_stopping=True,     # <- built-in early stopping
        validation_fraction=0.1,
        n_iter_no_change=20
    ))
])

scorers_hgb = {
    "roc_auc": "roc_auc",
    "accuracy": "accuracy",
    "precision": "precision",
    "recall": "recall",
    "f1": "f1"
}

hgb_cv = cross_validate(
    hgb_model, 
    x, y, 
    cv=cv, 
    scoring=scorers_hgb, 
    n_jobs=-1)

for m in scorers_hgb:
    s = hgb_cv[f"test_{m}"]
    print(f"{m.upper():<9}: {s.mean():.3f} (+/- {s.std()*2:.3f})")

MODEL 3: HIST-GRADIENT BOOSTING (EARLY STOP)


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ROC_AUC  : 0.673 (+/- 0.047)
ACCURACY : 0.981 (+/- 0.000)
PRECISION: 0.750 (+/- 0.775)
RECALL   : 0.018 (+/- 0.026)
F1       : 0.035 (+/- 0.050)


In [4]:
print("="*50)
print("MODEL 4: EXTRA TREES (PARALLEL & FAST)")
print("="*50)

et_model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("classifier", ExtraTreesClassifier(
        random_state=42,
        n_estimators=400,
        max_depth=None,          # let trees grow; try 20 if overfitting
        min_samples_split=5,
        min_samples_leaf=2,
        class_weight='balanced',
        n_jobs=-1,               # parallel
        bootstrap=False,         # typical for ExtraTrees
        max_features="sqrt"
    ))
])

scores_et = {
    "roc_auc": "roc_auc",
    "accuracy": "accuracy",
    "precision": "precision",
    "recall": "recall",
    "f1": "f1"
}

et_cv = cross_validate(
    et_model, 
    x, y, 
    cv=cv, 
    scoring=scores_et,
    n_jobs=-1)
    
for m in scores_et:
    s = et_cv[f"test_{m}"]
    print(f"{m.upper():<9}: {s.mean():.3f} (+/- {s.std()*2:.3f})")


MODEL 4: EXTRA TREES (PARALLEL & FAST)


ROC_AUC  : 0.699 (+/- 0.074)
ACCURACY : 0.940 (+/- 0.015)
PRECISION: 0.066 (+/- 0.039)
RECALL   : 0.155 (+/- 0.053)
F1       : 0.092 (+/- 0.046)


In [ ]:
print("="*50)
print("MODEL 5: XGBOOST (FAST)")
print("="*50)

xgb_model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("classifier", XGBClassifier(
        random_state=42,
        n_estimators=600,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method="hist",   # fast CPU histogram algorithm
        n_jobs=-1,
        reg_lambda=1.0,
        eval_metric="logloss"
    ))
])

scores_xgb = {
    "roc_auc":"roc_auc",
    "accuracy":"accuracy",
    "precision":"precision",
    "recall":"recall",
    "f1":"f1"
    }

xgb_cv = cross_validate(
    xgb_model, x, y, 
    cv=cv, 
    scoring=scores_xgb, 
    n_jobs=-1)

In [4]:
for m in scores_xgb:
    s = xgb_cv[f"test_{m}"] 
    print(f"{m.upper():<9}: {s.mean():.3f} (+/- {s.std()*2:.3f})")


ROC_AUC  : 0.635 (+/- 0.032)
ACCURACY : 0.981 (+/- 0.000)
PRECISION: 0.783 (+/- 0.389)
RECALL   : 0.023 (+/- 0.019)
F1       : 0.044 (+/- 0.036)


# 6. Model Training & Evaluation:

a. Train the selected model on the training set. <br>
b. Evaluate model performance using metrics suitable for imbalanced data (e.g., precision-recall curve, ROC-AUC score). <br>
c. Document the performance metrics, including confusion matrix, precision, recall, and F1-score. <br>

In [8]:
# Extra Trees Model - Train/Test Split (80/20) using existing model

print("="*50)
print("EXTRA TREES - TRAIN/TEST SPLIT (80/20)")
print("="*50)

# Split data into train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    x, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Training fraud rate: {y_train.mean()*100:.2f}%")
print(f"Test fraud rate: {y_test.mean()*100:.2f}%")

# Train the Extra Trees model on training data
print("\nTraining Extra Trees model on training data...")
et_model.fit(X_train, y_train)
print("Model training completed!")

# Make predictions on test data
y_pred = et_model.predict(X_test)
y_pred_proba = et_model.predict_proba(X_test)[:, 1]

# Create final_scores_best_model dictionary with all metrics
final_scores_best_model = {
    "roc_auc": roc_auc_score(y_test, y_pred_proba),
    "accuracy": accuracy_score(y_test, y_pred),
    "precision": precision_score(y_test, y_pred),
    "recall": recall_score(y_test, y_pred),
    "f1": f1_score(y_test, y_pred)
}

print(f"\nROC-AUC Score: {final_scores_best_model['roc_auc']:.3f}")

print(f"\nClassification Report:")
print(classification_report(y_test, y_pred))
print(f"\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


EXTRA TREES - TRAIN/TEST SPLIT (80/20)
Training set: 16399 samples
Test set: 4100 samples
Training fraud rate: 1.91%
Test fraud rate: 1.90%

Training Extra Trees model on training data...


Model training completed!

ROC-AUC Score: 0.694

Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      0.96      0.97      4022
         1.0       0.08      0.17      0.11        78

    accuracy                           0.95      4100
   macro avg       0.53      0.57      0.54      4100
weighted avg       0.97      0.95      0.96      4100


Confusion Matrix:
[[3881  141]
 [  65   13]]
